In [1]:
# Load libraries
import argparse
import logging
import os
import sys

import numpy as np
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm
# from tqdm.auto import tqdm
# from tqdm import tqdm_notebook as tqdm

from eval import eval_net
from unet import UNet

from torch.utils.tensorboard import SummaryWriter
from utils.dataset import BasicDataset
from torch.utils.data import DataLoader, random_split

In [2]:
# Paths
dir_img = 'data/train_images_256/'
dir_mask = 'data/train_masks_256/'
dir_checkpoint = 'checkpoints/'

In [3]:


def train_net(net,
              device,
              epochs=5,
              batch_size=8,
              lr=0.001,
              val_percent=0.1,
              save_cp=True,
              img_scale=0.5):

    dataset = BasicDataset(dir_img, dir_mask, img_scale)
    n_val = int(len(dataset) * val_percent)
    n_train = len(dataset) - n_val
    train, val = random_split(dataset, [n_train, n_val])
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=True)

    writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')
    global_step = 0

    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {batch_size}
        Learning rate:   {lr}
        Training size:   {n_train}
        Validation size: {n_val}
        Checkpoints:     {save_cp}
        Device:          {device.type}
        Images scaling:  {img_scale}
    ''')

    optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min' if net.n_classes > 1 else 'max', patience=2)
    if net.n_classes > 1:
        criterion = nn.CrossEntropyLoss()
    else:
        criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        net.train()

        epoch_loss = 0
        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
            for batch in train_loader:
                imgs = batch['image']
                true_masks = batch['mask']
                assert imgs.shape[1] == net.n_channels, \
                    f'Network has been defined with {net.n_channels} input channels, ' \
                    f'but loaded images have {imgs.shape[1]} channels. Please check that ' \
                    'the images are loaded correctly.'

                imgs = imgs.to(device=device, dtype=torch.float32)
                mask_type = torch.float32 if net.n_classes == 1 else torch.long
                # mask_type = torch.long
                true_masks = true_masks.to(device=device, dtype=mask_type)
                
                
                masks_softmax = net(imgs)
                
                # print(np.shape(masks_softmax),np.shape(true_masks))
                # true_masks[b,1, H, W], remove 1 to call cross_entropy_loss
                if len(np.shape(true_masks)) == 4:
                    loss = criterion(masks_softmax, true_masks.squeeze_(1)) 
                elif len(np.shape(true_masks)) == 3:                    
                    loss = criterion(masks_softmax, true_masks)
                else:
                    print("Minibatch size error")
                epoch_loss += loss.item()
                
                writer.add_scalar('Loss/train', loss.item(), global_step)

                pbar.set_postfix(**{'loss (batch)': loss.item()})

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_value_(net.parameters(), 0.1)
                optimizer.step()

                pbar.update(imgs.shape[0])
                global_step += 1
                if global_step % (len(dataset) // (10 * batch_size)) == 0:
                    for tag, value in net.named_parameters():
                        tag = tag.replace('.', '/')
                        writer.add_histogram('weights/' + tag, value.data.cpu().numpy(), global_step)
                        writer.add_histogram('grads/' + tag, value.grad.data.cpu().numpy(), global_step)
                    val_score = eval_net(net, val_loader, device)
                    scheduler.step(val_score)
                    writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], global_step)

                    if net.n_classes > 1:
                        logging.info('Validation cross entropy: {}'.format(val_score))
                        writer.add_scalar('Loss/test', val_score, global_step)
                    else:
                        logging.info('Validation Dice Coeff: {}'.format(val_score))
                        writer.add_scalar('Dice/test', val_score, global_step)

                    writer.add_images('images', imgs, global_step)
                    if net.n_classes == 1:
                        writer.add_images('masks/true', true_masks, global_step)
                        writer.add_images('masks/pred', torch.sigmoid(masks_pred) > 0.5, global_step)

        if save_cp:
            try:
                os.mkdir(dir_checkpoint)
                logging.info('Created checkpoint directory')
            except OSError:
                pass
            torch.save(net.state_dict(),
                       dir_checkpoint + f'CP_epoch{epoch + 1}.pth')
            logging.info(f'Checkpoint {epoch + 1} saved !')

    writer.close()

In [4]:



def get_args():
    parser = argparse.ArgumentParser(description='Train the UNet on images and target masks',
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('-e', '--epochs', metavar='E', type=int, default=5,
                        help='Number of epochs', dest='epochs')
    parser.add_argument('-b', '--batch-size', metavar='B', type=int, nargs='?', default=1,
                        help='Batch size', dest='batchsize')
    parser.add_argument('-l', '--learning-rate', metavar='LR', type=float, nargs='?', default=0.1,
                        help='Learning rate', dest='lr')
    parser.add_argument('-f', '--load', dest='load', type=str, default=False,
                        help='Load model from a .pth file')
    parser.add_argument('-s', '--scale', dest='scale', type=float, default=0.5,
                        help='Downscaling factor of the images')
    parser.add_argument('-v', '--validation', dest='val', type=float, default=10.0,
                        help='Percent of the data that is used as validation (0-100)')
    
    sys.argv[1:] = ['--epochs','200',  '--batch-size','1',  '--learning-rate','0.05',  
                    '--load', 'False',  '--scale','1.0', '--validation','10.0']
    # Or simply
    # sys.argv[1:] = ['-e','2',  '-b','4',  '-l','0.1', '-f', 'False',  '?-s','0.5', '-v','10.0']
    
    return parser.parse_args()

In [5]:
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
    args = get_args()
    print(args)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     device = torch.device( 'cpu')

    logging.info(f'Using device {device}')

    # Change here to adapt to your data
    # n_channels=1 for SAR data in this case, while 3 for RGB images
    # n_classes is the number of probabilities you want to get per pixel
    #   - For 1 class and background, use n_classes=1
    #   - For 2 classes, use n_classes=1
    #   - For N > 2 classes, use n_classes=N
    net = UNet(n_channels=1, n_classes=8, bilinear=True)
    logging.info(f'Network:\n'
                 f'\t{net.n_channels} input channels\n'
                 f'\t{net.n_classes} output channels (classes)\n'
                 f'\t{"Bilinear" if net.bilinear else "Transposed conv"} upscaling')

#     if args.load:
#         net.load_state_dict(
#             torch.load(args.load, map_location=device)
#         )
#         logging.info(f'Model loaded from {args.load}')

    net.to(device=device)
    # faster convolutions, but more memory
    # cudnn.benchmark = True

    try:
        train_net(net=net,
                  epochs=args.epochs,
                  batch_size=args.batchsize,
                  lr=args.lr,
                  device=device,
                  img_scale=args.scale,
                  val_percent=args.val / 100)
    except KeyboardInterrupt:
        torch.save(net.state_dict(), 'INTERRUPTED.pth')
        logging.info('Saved interrupt')
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)

INFO: Using device cuda
INFO: Network:
	1 input channels
	8 output channels (classes)
	Bilinear upscaling


Namespace(batchsize=1, epochs=200, load='False', lr=0.05, scale=1.0, val=10.0)


INFO: Creating dataset with 790 examples
I0530 01:26:01.395164 140247933708096 <ipython-input-3-e2ec59991c43>:29] Starting training:
        Epochs:          200
        Batch size:      1
        Learning rate:   0.05
        Training size:   711
        Validation size: 79
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1.0
    
Validation round: 100%|██████████| 79/79 [00:01<00:00, 50.13batch/s]
                                                                    I0530 01:26:10.081152 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 2.36281841905431
Validation round:  91%|█████████ | 72/79 [00:01<00:00, 40.45batch/s]
                                                                    I0530 01:26:18.628472 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.06022256600680279
Validation round:  91%|█████████ | 72/79 [00:01<00:00, 42.39batch/s]
                                                    

Validation round:  91%|█████████ | 72/79 [00:01<00:00, 39.31batch/s]
                                                                    I0530 01:27:11.736389 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.06277395683719078
Validation round:  91%|█████████ | 72/79 [00:01<00:00, 37.63batch/s]
                                                                    I0530 01:27:20.752524 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.0284662765423698
Epoch 1/200: 100%|██████████| 711/711 [01:19<00:00,  8.94img/s, loss (batch)=0.00241]
I0530 01:27:21.124857 140247933708096 <ipython-input-3-e2ec59991c43>:109] Checkpoint 1 saved !
Validation round: 100%|██████████| 79/79 [00:01<00:00, 44.53batch/s]
                                                                    I0530 01:27:30.663413 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.032301172422988585
Validation round:  91%|█████████ | 72/79 [00:01<00:

Validation round: 100%|██████████| 79/79 [00:01<00:00, 46.65batch/s]
                                                                    I0530 01:28:15.734829 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.02870163683681174
Validation round:  91%|█████████ | 72/79 [00:01<00:00, 38.90batch/s]
                                                                    I0530 01:28:24.514951 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.028262587722473413
Validation round:  91%|█████████ | 72/79 [00:01<00:00, 39.97batch/s]
                                                                    I0530 01:28:33.049998 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.02673784525430602
Validation round:  99%|█████████▊| 78/79 [00:01<00:00, 45.02batch/s]
                                                                    I0530 01:28:42.031677 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entr

...

Validation round:  91%|█████████ | 72/79 [00:01<00:00, 39.59batch/s]
                                                                    I0530 05:42:23.062884 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.02831470840907128
Validation round:  92%|█████████▏| 73/79 [00:01<00:00, 43.04batch/s]
                                                                    I0530 05:42:31.709327 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.03025743004298972
Validation round:  92%|█████████▏| 73/79 [00:01<00:00, 34.96batch/s]
                                                                    I0530 05:42:40.470977 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.026011578496372405
Validation round:  92%|█████████▏| 73/79 [00:01<00:00, 39.59batch/s]
                                                                    I0530 05:42:49.047134 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entr

Validation round:  92%|█████████▏| 73/79 [00:01<00:00, 41.13batch/s]
                                                                    I0530 05:43:23.610751 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.02609033259353895
Epoch 197/200: 100%|██████████| 711/711 [01:18<00:00,  9.07img/s, loss (batch)=0.00212]
I0530 05:43:23.898200 140247933708096 <ipython-input-3-e2ec59991c43>:109] Checkpoint 197 saved !
Validation round:  91%|█████████ | 72/79 [00:01<00:00, 37.44batch/s]
                                                                    I0530 05:43:32.969766 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.027313430335474367
Validation round:  92%|█████████▏| 73/79 [00:01<00:00, 40.57batch/s]
                                                                    I0530 05:43:41.674515 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.02590498070547923
Validation round:  91%|█████████ | 72/79 [00:0

Validation round: 100%|██████████| 79/79 [00:01<00:00, 47.50batch/s]
                                                                    I0530 05:44:33.536966 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.028212612874373422
Validation round:  91%|█████████ | 72/79 [00:01<00:00, 40.73batch/s]
                                                                    I0530 05:44:42.060810 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.027744585316063823
Epoch 198/200: 100%|██████████| 711/711 [01:18<00:00,  9.08img/s, loss (batch)=0.00218]
I0530 05:44:42.352328 140247933708096 <ipython-input-3-e2ec59991c43>:109] Checkpoint 198 saved !
Validation round:  91%|█████████ | 72/79 [00:01<00:00, 40.12batch/s]
                                                                    I0530 05:44:51.292777 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.0293435186095745
Validation round:  91%|█████████ | 72/79 [00:0

Validation round:  92%|█████████▏| 73/79 [00:01<00:00, 41.36batch/s]
                                                                    I0530 05:45:34.712416 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.03146535340433902
Validation round:  91%|█████████ | 72/79 [00:01<00:00, 39.46batch/s]
                                                                    I0530 05:45:43.403003 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.027810776973499835
Validation round:  92%|█████████▏| 73/79 [00:01<00:00, 40.69batch/s]
                                                                    I0530 05:45:52.097110 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.026849180401584335
Validation round:  96%|█████████▌| 76/79 [00:01<00:00, 45.31batch/s]
                                                                    I0530 05:46:00.724521 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross ent

Validation round:  91%|█████████ | 72/79 [00:01<00:00, 41.67batch/s]
                                                                    I0530 05:46:44.976615 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.02641496474540032
Validation round:  92%|█████████▏| 73/79 [00:01<00:00, 33.46batch/s]
                                                                    I0530 05:46:53.806743 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.02704670086686862
Validation round:  92%|█████████▏| 73/79 [00:01<00:00, 38.11batch/s]
                                                                    I0530 05:47:03.466508 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entropy: 0.027368917676643765
Validation round:  92%|█████████▏| 73/79 [00:01<00:00, 39.93batch/s]
                                                                    I0530 05:47:12.271143 140247933708096 <ipython-input-3-e2ec59991c43>:90] Validation cross entr

In [ ]:
one_hot.dim()